In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
import tensorflow as tf
import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras import backend as K
from tensorflow.keras.models import clone_model
from keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras import models, layers, optimizers, losses
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D

In [3]:
# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype("float32") / 255.0
x_train = np.reshape(x_train, (-1, 32, 32, 3))

x_test = x_test.astype("float32") / 255.0
x_test = np.reshape(x_test, (-1, 32, 32, 3))

In [4]:

# Define the AlexNet model
model = Sequential()

model.add(Conv2D(96, (11, 11), input_shape=(32, 32, 3), padding='same'))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

model.add(ZeroPadding2D(padding=(2, 2)))

model.add(Conv2D(256, (5, 5), padding='same'))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

model.add(ZeroPadding2D(padding=(1, 1)))

model.add(Conv2D(384, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(ZeroPadding2D(padding=(1, 1)))

model.add(Conv2D(384, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(ZeroPadding2D(padding=(1, 1)))

model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

model.add(Flatten())

model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(1000))
model.add(Activation('softmax'))


In [ ]:
# Compile the model
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
              optimizer=SGD(learning_rate=0.001), 
              metrics=[keras.metrics.SparseCategoricalAccuracy()])

# Train the model
model.fit(x_train, y_train, batch_size=32, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100


/usr/local/lib/python3.9/dist-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1563/1563 [==============================] - 68s 38ms/step - loss: 3.0732 - sparse_categorical_accuracy: 0.1147 - val_loss: 2.2393 - val_sparse_categorical_accuracy: 0.1784
Epoch 2/100
1563/1563 [==============================] - 61s 39ms/step - loss: 2.3145 - sparse_categorical_accuracy: 0.1475 - val_loss: 2.1907 - val_sparse_categorical_accuracy: 0.1947
Epoch 3/100
1563/1563 [==============================] - 60s 39ms/step - loss: 2.1823 - sparse_categorical_accuracy: 0.1862 - val_loss: 2.0123 - val_sparse_categorical_accuracy: 0.2604
Epoch 4/100
1563/1563 [==============================] - 62s 40ms/step - loss: 2.0375 - sparse_categorical_accuracy: 0.2405 - val_loss: 1.9230 - val_sparse_categorical_accuracy: 0.3057
Epoch 5/100
1563/1563 [==============================] - 61s 39ms/step - loss: 1.9590 - sparse_categorical_accuracy: 0.2787 - val_loss: 1.8543 - val_sparse_categorical_accuracy: 0.3267
Epoch 6/100
1563/1563 [==============================] - 61s 39ms/step - loss: 1.8715 -

In [ ]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)

# Print the test accuracy
print(f'Test accuracy: {test_acc}')

In [ ]:
# Save the trained model
model.save('/content/drive/MyDrive/arch1-cifar10.h5')

In [ ]:
quantized_model=tf.lite.TFLiteConverter.from_saved_model('/content/drive/MyDrive/arch1-cifar10.h5')
quantized_model.optimizations=[tf.lite.Optimize.DEFAULT]
tf_quantized_model=quantized_model.convert()

In [ ]:
with open("quantized_model.tflite","wb") as f:
  f.write(tf_quantized_model)

In [ ]:
import numpy as np
import tensorflow as tf
from keras.datasets import cifar10
from keras.utils import to_categorical

# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Preprocess the data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="/content/quantized_model.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Evaluate the model on the test set
correct = 0
total = 0
for i in range(len(x_test)):
    input_data = np.expand_dims(x_test[i], axis=0).astype(input_details[0]['dtype'])
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    predicted_class = np.argmax(output_data)
    true_class = np.argmax(y_test[i])
    if predicted_class == true_class:
        correct += 1
    total += 1

accuracy = correct / total
print("Test accuracy:", accuracy)
